In [1]:
from inventory import Inventory
from exception import InventoryRemoveZeroError, InventoryRemoveExcessiveError
from entity import Entity
from sql import select_inventory, select_entity
from uuid import uuid4

In [2]:
# coonect using duckdb and validate tables
import duckdb as dd

# Create an in-memory DuckDB connection
con = dd.connect(':memory:')

# dd.sql("SELECT * FROM sqlite_scan('../data/inventory.db', 'inventory_sql');").show()
try:
    dd.sql("ATTACH '../data/inventory.db' AS inventory (TYPE sqlite);")
except:
    print("already attached")
dd.sql("USE inventory")

already attached


In [3]:
# define database queries
sql_inventory = '''
        select INV.hierarchy_level as lvl
            , INV.quantity as qty
            , CHI.description_singular as child
            , INV.position as pos
            , PAR.description_singular as parent
            , INV.inventory_id
            , INV.inventory_id_parent
            , PAR.entity_id as parent
            , CHI.entity_id as child
        from inventory_sql INV
        inner join entity_sql PAR on PAR.entity_id = INV.entity_id_parent
        inner join entity_sql CHI on CHI.entity_id = INV.entity_id_child
    '''

In [4]:
# try adding a yellow chair to the lounge
item = Entity(description="Yellow Chair")
print(item)
location = Entity(description="Lounge")
print(location)

Yellow Chair (total inventory: 3) (4d10faad-8a10-436b-b732-011a7b4ad043)
Lounge (total inventory: 1) (03dd5c43-d66e-4c3f-a6eb-eb67b47c0373)


In [5]:
# check the inventory for the item
print(f"{item.description} inventory:")
sql_where = f"WHERE INV.entity_id_child = '{item.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()
# and the inventory for the location
print(f"{location.description} inventory:")
sql_where = f"WHERE INV.entity_id_parent = '{location.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

yellow chair inventory:
┌───────┬───────┬──────────────┬─────────┬─────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │   parent    │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │   varchar   │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     3 │ yellow chair │ IN      │ dining room │ 4e18e23a-ca8b-4ed3-9ba6-8e2fb46d6cd8 │ 6dc856f4-42fe-4fe0-868b-9d00

In [6]:
# try adding 4 yellow chairs to the lounge
inventory = Inventory(entity_parent=location, entity_child=item, quantity=4)
print(inventory)
sql_where = f"WHERE INV.entity_id_parent = '{location.entity_id}' and entity_id_child = '{item.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

Inventory created successfully
4 yellow chairs (4d10faad-8a10-436b-b732-011a7b4ad043) in lounge (03dd5c43-d66e-4c3f-a6eb-eb67b47c0373)
┌───────┬───────┬──────────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     4 │ yellow chai

In [7]:
# check the inventory for the item
print(f"{item.description} inventory:")
sql_where = f"WHERE INV.entity_id_child = '{item.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()
# and the inventory for the location
print(f"{location.description} inventory:")
sql_where = f"WHERE INV.entity_id_parent = '{location.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

yellow chair inventory:
┌───────┬───────┬──────────────┬─────────┬─────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │   parent    │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │   varchar   │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     4 │ yellow chair │ IN      │ lounge      │ bea55c6e-4313-4f89-8b59-e579b17d32e7 │ 8ce8d635-c5c5-49d7-bdcf-2b5c

In [8]:
# try adding a further 2 yellow chairs to the lounge
inventory = Inventory(entity_parent=location, entity_child=item, quantity=2)
print(inventory)
sql_where = f"WHERE INV.entity_id_parent = '{location.entity_id}' and entity_id_child = '{item.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

Inventory updated successfully
6 yellow chairs (4d10faad-8a10-436b-b732-011a7b4ad043) in lounge (03dd5c43-d66e-4c3f-a6eb-eb67b47c0373)
┌───────┬───────┬──────────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     6 │ yellow chai

In [9]:
# try removing too many yellow chairs from the lounge
try:
    inventory = Inventory(entity_parent=location, entity_child=item, quantity=99999, action="REMOVE")
except InventoryRemoveExcessiveError as e:
    print(e)

There are not enough yellow chairs in lounge to remove


In [10]:
# remove all yellow chairs from lounge
new_quantity = inventory._quantity_new
inventory = Inventory(entity_parent=location, entity_child=item, quantity=new_quantity, action="REMOVE")
# inventory record should have been deleted
sql_where = f"WHERE INV.entity_id_parent = '{location.entity_id}' and entity_id_child = '{item.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

Inventory deleted successfully
┌───────┬───────┬─────────┬─────────┬─────────┬──────────────┬─────────────────────┬─────────┬─────────┐
│  lvl  │  qty  │  child  │   pos   │ parent  │ inventory_id │ inventory_id_parent │ parent  │  child  │
│ int64 │ int64 │ varchar │ varchar │ varchar │   varchar    │       varchar       │ varchar │ varchar │
├───────┴───────┴─────────┴─────────┴─────────┴──────────────┴─────────────────────┴─────────┴─────────┤
│                                                0 rows                                                │
└──────────────────────────────────────────────────────────────────────────────────────────────────────┘



In [11]:
# try removing a further yellow chair from the lounge
try:
    inventory = Inventory(entity_parent=location, entity_child=item, quantity=1, action="REMOVE")
except InventoryRemoveZeroError as e:
    print(e)

There are no yellow chairs in lounge to remove


In [12]:
# add a new level 1 location
location1 = Entity(description="Stillwaters",base_hierarchy_level = 1, entity_id="339cfbbb-f263-41f0-b4dc-5abcfe83e98c")
dd.sql(f"select * from entity_sql where entity_key = 'stillwaters'").show()

Entity created successfully: Stillwaters (total inventory: 0) (339cfbbb-f263-41f0-b4dc-5abcfe83e98c)
┌───────────┬──────────────────┬────────────┬─────────┬────────────┬──────────────────────┬────────────────────┬──────────────────────┬──────────────┐
│ entity_id │ entity_id_parent │ entity_key │  noun   │ adjectives │ description_singular │ description_plural │ base_hierarchy_level │ created_time │
│  varchar  │     varchar      │  varchar   │ varchar │  varchar   │       varchar        │      varchar       │        int64         │  timestamp   │
├───────────┴──────────────────┴────────────┴─────────┴────────────┴──────────────────────┴────────────────────┴──────────────────────┴──────────────┤
│                                                                       0 rows                                                                       │
└───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [13]:
# add a new location
location2 = Entity(description="Dungeon", entity_id="9388641c-36bb-42f8-92b5-0d267098e7f1")
dd.sql(f"select * from entity_sql where entity_key = 'dungeon'").show()

Entity created successfully: Dungeon (total inventory: 0) (9388641c-36bb-42f8-92b5-0d267098e7f1)
┌──────────────────────────────────────┬──────────────────┬────────────┬─────────┬────────────┬──────────────────────┬────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │ entity_key │  noun   │ adjectives │ description_singular │ description_plural │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │  varchar   │ varchar │  varchar   │       varchar        │      varchar       │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼────────────┼─────────┼────────────┼──────────────────────┼────────────────────┼──────────────────────┼─────────────────────┤
│ 9388641c-36bb-42f8-92b5-0d267098e7f1 │ None             │ dungeon    │ dungeon │            │ dungeon              │ dungeons           │                    0 │ 

In [14]:
# create an inventory record for the new level 1 location
inventory = Inventory(entity_parent=location1, entity_child=location2, quantity=1)
print(location1.entity_id, location2.entity_id)
# inventory record newly created
sql_where = f"WHERE INV.entity_id_parent = '{location1.entity_id}' and entity_id_child = '{location2.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

Inventory created successfully
339cfbbb-f263-41f0-b4dc-5abcfe83e98c 9388641c-36bb-42f8-92b5-0d267098e7f1
┌───────┬───────┬─────────┬─────────┬─────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │  child  │   pos   │   parent    │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │ varchar │ varchar │   varchar   │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼─────────┼─────────┼─────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     1 │     1 │ dungeon │ IN      │ stillwaters │ c01a1f9b-a9

In [15]:
# add a cd rack to the dungeon
item = Entity("CD Rack")
location = Entity("Dungeon")
inventory = Inventory(entity_parent=location, entity_child=item, quantity=1)
# inventory record newly created
sql_where = f"WHERE INV.entity_id_parent = '{location.entity_id}' and entity_id_child = '{item.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

Entity created successfully: Cd Rack (total inventory: 0) (ea4731c1-c66a-49d2-be8c-ceb335ac34f1)
Inventory created successfully
┌───────┬───────┬─────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │  child  │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │ varchar │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼─────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     1 │ cd rack │ IN      │ dungeon │ 90d9157f

In [16]:
# add a cd to the cd rack
item = Entity("The World Wont Listen by The Smiths")
location = Entity("CD Rack")
inventory = Inventory(entity_parent=location, entity_child=item, quantity=1)
# inventory record newly created
sql_where = f"WHERE INV.entity_id_parent = '{location.entity_id}' and entity_id_child = '{item.entity_id}'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

Entity created successfully: World Wont Listen By Smith (total inventory: 0) (fe214a8b-f45d-40f8-bdbe-36f869685436)
Inventory created successfully
┌───────┬───────┬────────────────────────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │           child            │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │          varchar           │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼────────────────────────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼─

In [17]:
# add a lounge to the stillwaters location
inventory = Inventory(entity_parent=Entity("stillwaters"), entity_child=Entity("lounge"), quantity=1)
dd.sql(f"select * from entity_sql where entity_key = 'lounge'").show()

Inventory created successfully
┌──────────────────────────────────────┬──────────────────┬────────────┬─────────┬────────────┬──────────────────────┬────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │ entity_key │  noun   │ adjectives │ description_singular │ description_plural │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │  varchar   │ varchar │  varchar   │       varchar        │      varchar       │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼────────────┼─────────┼────────────┼──────────────────────┼────────────────────┼──────────────────────┼─────────────────────┤
│ 03dd5c43-d66e-4c3f-a6eb-eb67b47c0373 │ None             │ lounge     │ lounge  │            │ lounge               │ lounges            │                    0 │ 2025-03-10 12:45:38 │
└──────────────────────────────────────┴────

In [18]:
# add a sofa to the lounge
inventory = Inventory(entity_parent=Entity("lounge"), entity_child=Entity("sofa"), quantity=1)

Entity created successfully: Sofa (total inventory: 0) (14b0a5f2-db16-4718-a7ef-76ad28b47ed9)
┌───────┬───────┬─────────┬─────────┬───────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │  child  │   pos   │      parent       │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │ varchar │ varchar │      varchar      │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼─────────┼─────────┼───────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     1 │     1 │ lounge  │ IN      │ st chads vic

AttributeError: 'Inventory' object has no attribute 'hierarchy_level'

In [ ]:
dd.close()